# Ejercicio 6: Dense Retrieval e Introducción a FAISS

## Objetivo de la práctica

Generar embeddings con sentence-transformers (SBERT, E5), e indexar documentos con FAISS

## Parte 0: Carga del Corpus
### Actividad

1. Carga el corpus 20 Newsgroups desde sklearn.datasets.fetch_20newsgroups.
2. Limita el corpus a los primeros 2000 documentos para facilitar el procesamiento.

In [22]:
# Carga del corpus 20 Newsgroups.
from sklearn.datasets import fetch_20newsgroups

In [23]:
# Limitar el corpus a los primeros 2000 documentos.
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
newsgroupsdocs = newsgroups.data[:2000]

## Parte 2: Generación de Embeddings
### Actividad

1. Usa dos modelos de sentence-transformers. Puedes usar: `'all-MiniLM-L6-v2'` (SBERT), o `'intfloat/e5-base'` (E5). Cuando uses E5, antepon `"passage: "` a cada documento antes de codificar.
2. Genera los vectores de embeddings para todos los documentos usando el modelo seleccionado.
3. Guarda los embeddings en un array de NumPy para su posterior indexación.

In [24]:
# Usar el modelo sentence-transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
# Genera los vectores de embeddings para todos los documentos usando el modelo seleccionado.
embeddings = model.encode(newsgroupsdocs)

In [25]:
import numpy as np
import pandas as pd
# Generar una estructura de datos con id, newsgroupsdocs, doc_embedding
data = []
for idx, (doc, embedding) in enumerate(zip(newsgroupsdocs, embeddings)):
    data.append((idx, doc, embedding))
# Convertir a DataFrame para visualización clara
df = pd.DataFrame(data, columns=['id', 'doc', 'doc-embedding'])
print(df.head(10))

   id                                                doc  \
0   0  \n\nI am sure some bashers of Pens fans are pr...   
1   1  My brother is in the market for a high-perform...   
2   2  \n\n\n\n\tFinally you said what you dream abou...   
3   3  \nThink!\n\nIt's the SCSI card doing the DMA t...   
4   4  1)    I have an old Jasmine drive which I cann...   
5   5  \n\nBack in high school I worked as a lab assi...   
6   6  \n\nAE is in Dallas...try 214/241-6060 or 214/...   
7   7  \n[stuff deleted]\n\nOk, here's the solution t...   
8   8  \n\n\nYeah, it's the second one.  And I believ...   
9   9  \nIf a Christian means someone who believes in...   

                                       doc-embedding  
0  [0.0020779776, 0.02345035, 0.024808845, -0.010...  
1  [0.0500603, 0.026980925, -0.008864761, -0.0359...  
2  [0.016404761, 0.08100052, -0.049535934, -0.008...  
3  [-0.019391418, 0.011494355, -0.014787201, -0.0...  
4  [-0.039287027, -0.055402838, -0.074536085, -0....  
5  [0.021

In [26]:
# Guardar los embeddings en un array para luego ser indexados
doc_embeddings = np.array(embeddings)

## Parte 3: Indexación con FAISS
### Actividad

1. Crea un índice plano con faiss.IndexFlatL2 para búsquedas por distancia euclidiana.
2. Asegúrate de usar la dimensión correcta `(embedding_dim = doc_embeddings.shape[1])`.
3. Agrega los vectores de documentos al índice.

In [27]:
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [28]:
import faiss
# Vectores de documentos.
embedding_dim = doc_embeddings.shape[1]
# Índice de  para búsquedas por distancia euclidiana
index = faiss.IndexFlatL2(embedding_dim)
# Añadir los embeddings al índice
index.add(doc_embeddings)

## Parte 4: Consulta Semántica
### Actividad

1. Escribe una consulta en lenguaje natural. Ejemplos:

    * "God, religion, and spirituality"
    * "space exploration"
    * "car maintenance"

2. Codifica la consulta utilizando el mismo modelo de embeddings. Cuando uses E5, antepon `"query: "` a la consulta.
3. Recupera los 5 documentos más relevantes con `index.search(...)`.
4. Muestra los textos de los documentos recuperados (puedes mostrar solo los primeros 500 caracteres de cada uno).

In [29]:
query = "space exploration"
# Codifica la consulta utilizando el mismo modelo de embeddings.
query_embedding = model.encode([query])

In [31]:
# Recuperación de los 5 documentos más relevantes.
k = 5
distances, indices = index.search(query_embedding, k)

In [32]:
# Muestra los textos de los documentos recuperados, mostrando los primeros 500 caracteres de cada uno
for i in range(k):
    doc_id = indices[0][i]
    print(f"Documento {doc_id} (distancia: {distances[0][i]:.4f}):")
    print(newsgroupsdocs[doc_id][:500])  # Muestra los primeros 500 caracteres del documento
    print("\n" + "="*80 + "\n")  # Separador entre documentos

Documento 495 (distancia: 1.0018):
I am posting this for a friend without internet access. Please inquire
to the phone number and address listed.
---------------------------------------------------------------------

"Space: Teaching's Newest Frontier"
Sponsored by the Planetary Studies Foundation

The Planetary Studies Foundation is sponsoring a one week class for
teachers called "Space: Teaching's Newest Frontier." The class will be
held at the Sheraton Suites in Elk Grove, Illinois from June 14 through
June 18. Participants wh


Documento 1643 (distancia: 1.1204):

Well, here goes.

The first item of business is to establish the importance space life
sciences in the whole of scheme of humankind.  I mean compared
to football and baseball, the average joe schmoe doesn't seem interested
or even curious about spaceflight.  I think that this forum can
make a major change in that lack of insight and education.

All of us, in our own way, can contribute to a comprehensive document
which ca